# Technical Assessment (SQL)

1.	Using the tables above, write a query that returns, for each order, the order_id, the product name, the difference in days between the product go live date and the ordered at date, and the marketing ad network and source. This result should be five rows. Also include the output of the query. 

In [ ]:
'''
SELECT 
  o.id AS order_id, 
  p.name AS product_name, 
  (
    CAST(o.ordered_at AS DATE) - p.go_live_date
  ) AS "days between live date and date ordered", 
  m.ad_network, 
  m.source 
FROM orders o 
  JOIN products p ON o.product_id = p.id 
  JOIN marketing_orders mo ON mo.order_id = o.id 
  JOIN marketing m ON m.id = mo.marketing_id;
'''

<a>
	<img src="ques_1.png">
</a>

2.	Using the tables above, write a query that returns the best performing marketing ad network and source, a count of how many orders it had, and the best selling product for that ad network and source. Also include the output of the query.
Hint: The best selling product will have to be retrieved through seeing which orders are associated with the best performing marketing ad network and source. In this case, best performing means “Has the largest number of records in the DB associated with it”. This result will be a single row.

In [ ]:
'''
WITH ad_source_info AS (
SELECT m.ad_network, 
       m.source, 
       p.id AS prod_id, 
       p.name AS product_name, 
       CAST(o.product_quantity AS INT), 
        COUNT(o.id) OVER (
            PARTITION BY m.id ORDER BY CAST (o.product_quantity AS INT) DESC
                RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
            ) AS order_cnt	
FROM orders o 
  JOIN products p ON o.product_id = p.id 
  JOIN marketing_orders mo ON mo.order_id = o.id 
  JOIN marketing m ON m.id = mo.marketing_id
)
SELECT ad_network, 
        source, 
        order_cnt, 
        product_name 
FROM ad_source_info
GROUP BY ad_network, source, prod_id, product_name, order_cnt
ORDER BY order_cnt DESC, SUM(product_quantity) DESC
LIMIT 1; 
'''

<a>
	<img src="ques_2.png">
</a>

3.	Write a query that determines the best selling product for each month of orders. Also include the output of the query.

In [ ]:
'''
WITH mth_prod_info AS (
  SELECT 
    EXTRACT(MONTH FROM ordered_at) AS month, 
    p.name AS product_name, 
    RANK() OVER (
      PARTITION BY EXTRACT(MONTH FROM ordered_at 
      	) ORDER BY CAST(o.product_quantity AS INT) DESC
    ) AS ranking
  FROM orders o 
    JOIN products p ON o.product_id = p.id 
  ORDER BY 
    month
) 
SELECT 
  month, 
  product_name 
FROM mth_prod_info 
WHERE ranking = 1;
'''

<a>
	<img src="ques_3.png">
</a>